In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
import warnings
warnings.filterwarnings('ignore')


def est_ndwcov_factor(Y, factors, ic, lambda_min=True):
    """
    Estimate nodewise covariance with factor models using LASSO.
    
    Parameters:
    -----------
    Y : numpy.ndarray
        n x p matrix of observations
    factors : numpy.ndarray
        n x k matrix of factors
    ic : str
        Information criterion: 'WIC', 'BIC', 'GIC', 'AIC', or 'cv'
    lambda_min : bool
        If True and ic='cv', use lambda.min; otherwise use lambda.1se
        
    Returns:
    --------
    TAU : numpy.ndarray
        p x p precision matrix estimate
    """
    # Initialization
    p = Y.shape[1]
    n = Y.shape[0]
    C = np.zeros((p, p))
    np.fill_diagonal(C, 1)
    tau = []
    ns1 = np.ones((n, 1))
    
    # Fit factor model: Y = factors * beta + u
    # Add intercept to factors
    factors_with_intercept = np.column_stack([np.ones(n), factors])
    
    # Fit linear regression for each column of Y
    factormodel = LinearRegression(fit_intercept=False)
    factormodel.fit(factors_with_intercept, Y)
    
    # Get residuals and beta coefficients (excluding intercept)
    u = Y - factormodel.predict(factors_with_intercept)
    beta = factormodel.coef_[:, 1:]  # p x k matrix (excluding intercept)
    
    # Loop over the assets
    for j in range(p):
        # Create design matrix excluding column j
        X_j = np.delete(u, j, axis=1)
        y_j = u[:, j]
        
        if ic != 'cv':
            # Fit LASSO path
            alphas = np.logspace(-4, 1, 100)  # Create lambda sequence
            df_list = []
            sig_list = []
            bic_list = []
            coef_list = []
            res_list = []
            
            for alpha in alphas:
                model = Lasso(alpha=alpha, fit_intercept=False, max_iter=10000)
                model.fit(X_j, y_j)
                
                # Predictions and residuals
                y_pred = model.predict(X_j)
                res = y_j - y_pred
                
                # Degrees of freedom (number of non-zero coefficients)
                df = np.sum(np.abs(model.coef_) > 1e-8)
                
                # Variance of residuals
                sig = np.sum(res**2) / n
                
                # Compute information criterion
                if ic == 'WIC':
                    bic_val = np.log(sig) + df * np.log(n) / n * np.log(np.log(p))
                elif ic == 'BIC':
                    bic_val = np.log(sig) + df * np.log(n) / n
                elif ic == 'GIC':
                    bic_val = np.log(sig) + df * np.log(p) * np.log(np.log(n)) / n
                elif ic == 'AIC':
                    bic_val = np.log(sig) + 2 * df
                else:
                    raise ValueError(f"Unknown IC: {ic}")
                
                df_list.append(df)
                sig_list.append(sig)
                bic_list.append(bic_val)
                coef_list.append(model.coef_.copy())
                res_list.append(res)
            
            # Select model with minimum IC
            jind = np.argmin(bic_list)
            jpar = coef_list[jind]
            jres = res_list[jind]
            jtau = np.sum(y_j * jres) / n
            
        else:  # Cross-validation
            lasso_cv = LassoCV(cv=5, fit_intercept=False, max_iter=10000, n_alphas=100)
            lasso_cv.fit(X_j, y_j)
            
            if lambda_min:
                # Use alpha that minimizes CV error (lambda.min equivalent)
                jfit = lasso_cv.predict(X_j)
                jpar = lasso_cv.coef_
            else:
                # Use alpha within 1 SE of minimum (lambda.1se equivalent)
                cv_scores = lasso_cv.mse_path_.mean(axis=1)
                cv_std = lasso_cv.mse_path_.std(axis=1)
                min_idx = np.argmin(cv_scores)
                threshold = cv_scores[min_idx] + cv_std[min_idx]
                
                # Find largest alpha with CV score below threshold
                valid_indices = np.where(cv_scores <= threshold)[0]
                se_idx = valid_indices[0] if len(valid_indices) > 0 else min_idx
                
                selected_alpha = lasso_cv.alphas_[se_idx]
                model_1se = Lasso(alpha=selected_alpha, fit_intercept=False, max_iter=10000)
                model_1se.fit(X_j, y_j)
                jfit = model_1se.predict(X_j)
                jpar = model_1se.coef_
            
            jres = y_j - jfit
            jtau = np.sum(y_j * jres) / n
        
        # Fill in C matrix
        # Insert coefficients back (accounting for missing j-th position)
        C_row = np.insert(-jpar / jtau, j, 0)
        C[j, :] = C_row
        tau.append(jtau)
    
    # Set diagonal
    np.fill_diagonal(C, 1 / np.array(tau))
    omega = C.copy()
    omegasym = (C + C.T) / 2
    
    # Compute factor covariance - ensure float64
    covft = (1/n) * (factors.T @ factors) - (1/(n**2)) * (factors.T @ ns1 @ ns1.T @ factors)
    covft = covft.astype(np.float64)
    
    # Ensure beta and omegasym are float64
    beta = beta.astype(np.float64)
    omegasym = omegasym.astype(np.float64)
    
    # Compute TAU
    if factors.shape[1] == 1:
        covft_inv = 1.0 / float(covft[0, 0])
        p1 = 1.0 / (covft_inv + beta.T @ omegasym @ beta)
        TAU = omega - omega @ beta @ p1 @ beta.T @ omega
    else:
        covft_inv = np.linalg.inv(covft)
        p1 = np.linalg.inv(covft_inv + beta.T @ omegasym @ beta)
        TAU = omega - omega @ beta @ p1 @ beta.T @ omega
    
    return TAU


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star


def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star


def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star


def load_yearly_signals(year, buys_path_template='buys_{}.csv', sells_path_template='sells_{}.csv'):
    """
    Load buy and sell signals for a specific year.
    
    Parameters:
    -----------
    year : int
        Year to load signals for
    buys_path_template : str
        Template for buys file path (use {} for year placeholder)
    sells_path_template : str
        Template for sells file path (use {} for year placeholder)
    
    Returns:
    --------
    permno_set : set
        Set of permnos in the buy and sell signals for this year
    """
    try:
        buys = pd.read_csv(buys_path_template.format(year), index_col=1)
        sells = pd.read_csv(sells_path_template.format(year), index_col=1)
        
        buys.index.name = 'permno'
        sells.index.name = 'permno'
        
        buys_index = buys.index.astype(int)
        sells_index = sells.index.astype(int)
        
        return set(buys_index.union(sells_index))
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load signals for year {year}: {e}")
        return set()


def load_ff_factors(factors_path='factors_ff_monthly_raw.csv'):
    """
    Load Fama-French factors from CSV file.
    
    Parameters:
    -----------
    factors_path : str
        Path to the factors CSV file
    
    Returns:
    --------
    factors_df : pd.DataFrame
        DataFrame with date index and factor columns
    """
    factors_df = pd.read_csv(factors_path)
    
    # Convert month column (e.g., 192707) to datetime
    # This gives us the first day of the month (1927-07-01)
    factors_df['date'] = pd.to_datetime(factors_df.iloc[:, 0].astype(str), format='%Y%m')
    
    # Convert to end of month to match returns data
    factors_df['date'] = factors_df['date'] + pd.offsets.MonthEnd(0)
    
    # Set date as index and keep only factor columns
    factors_df = factors_df.set_index('date')[['Mkt-RF', 'SMB', 'HML']]
    
    # Convert to decimal form (assuming factors are in percentage points)
    factors_df = factors_df / 100
    
    return factors_df


def calculate_exit_transaction_cost(prev_weights_dict, prev_oos_returns_dict, 
                                    prev_gross_return, transaction_cost, verbose=False):
    """
    Calculate transaction cost when exiting the market (liquidating all positions).
    """
    if len(prev_weights_dict) == 0:
        return 0.0, 0.0, 0.0
    
    # Adjust previous weights to current period's BEGINNING
    adjusted_prev = {}
    for asset, prev_w in prev_weights_dict.items():
        if asset in prev_oos_returns_dict:
            prev_r = prev_oos_returns_dict[asset]
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
        else:
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
    
    # Turnover (Selling everything to Cash)
    turnover = sum(abs(w) for w in adjusted_prev.values())
    
    # Cost 
    tc = transaction_cost * 1.0 * turnover
    
    # Net Return is 0.0 (Cash return) - Cost
    net_return = -tc
    
    if verbose:
        print(f"  Liquidating positions | Turnover: {turnover:>6.4f} | TC: {tc:>8.6f}")
    
    return turnover, tc, net_return


def backtest_factor_nodewise_yearly(df, 
                                     factors_path='factors_ff_monthly_raw.csv',
                                     test_start_date='2020-01-31', 
                                     test_end_date='2024-11-30',
                                     lookback_window=180,
                                     transaction_cost=0.001,
                                     buys_path_template='buys_{}.csv',
                                     sells_path_template='sells_{}.csv',
                                     portfolio_types=['gmv', 'mv', 'msr'],
                                     mv_target_return=0.01,
                                     ic='GIC',
                                     verbose=True):
    """
    Backtest Factor-based Nodewise regression with multiple portfolio types.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    factors_path : str
        Path to Fama-French factors CSV file
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.001 = 10 bps)
    buys_path_template : str
        Template for buys file path (use {} for year placeholder)
    sells_path_template : str
        Template for sells file path (use {} for year placeholder)
    portfolio_types : list of str
        Portfolio types to compute: 'gmv', 'mv', 'msr' (default: all three)
    mv_target_return : float
        Target return for MV portfolio (default: 0.01 = 1% monthly)
    ic : str
        Information criterion for LASSO: 'WIC', 'BIC', 'GIC', 'AIC', or 'cv'
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    all_results : dict
        Dictionary with keys as portfolio types, values as (results_df, metrics) tuples
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])
    
    # Load Fama-French factors
    factors_df = load_ff_factors(factors_path)
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Initialize storage for each portfolio type
    portfolio_data = {ptype: {
        'returns': [],
        'dates': [],
        'weights_list': [],
        'turnover_list': [],
        'gross_returns': [],
        'prev_weights_dict': {},
        'prev_oos_returns_dict': {},
        'prev_gross_return': 0.0
    } for ptype in portfolio_types}
    
    # Cache for yearly signals
    yearly_signals_cache = {}
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print(f"STARTING FACTOR-BASED BACKTEST: {', '.join(portfolio_types).upper()}")
        print(f"Information Criterion: {ic}")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        current_year = current_date.year
        
        # Load signals for current year if not cached
        if current_year not in yearly_signals_cache:
            yearly_signals_cache[current_year] = load_yearly_signals(
                current_year, buys_path_template, sells_path_template
            )
        
        allowed_permnos = yearly_signals_cache[current_year]
        
        # Get OOS returns FIRST
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Handle early exit cases
        if len(allowed_permnos) == 0:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ No signals for year {current_year}, recording zero return")
            
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
            continue
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        
        # Pivot to get returns matrix
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        
        # Reindex to ensure all dates are present
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)
        
        # Align factors with return realization dates
        factor_dates = [(d + pd.DateOffset(months=1) + pd.offsets.MonthEnd(0)) for d in window_dates]
        
        try:
            factors_window = factors_df.loc[factor_dates]
        except KeyError:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ Missing factor data, recording zero return")
            
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
            continue
        
        if factors_window.isna().any().any():
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ Missing factor data in window, recording zero return")
            
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
            continue
        
        # Filter assets with NaNs
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        factors = factors_window.values
        n_train, p_current = Y.shape

        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')} | Year: {current_year}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f"  Signals: {len(allowed_permnos)} permnos | Assets: {p_current}")

        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), recording zero return")
            
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
            continue
        
        try:
            if verbose:
                print(f"  Running Factor-based Nodewise Regression (IC={ic})...")
            
            # Run factor-based nodewise regression
            Theta_hat = est_ndwcov_factor(Y, factors, ic=ic, lambda_min=True)
            
            # Compute weights for each portfolio type
            weights_dict_by_type = {}
            
            if 'gmv' in portfolio_types:
                if verbose:
                    print(f"  Computing GMV weights...")
                w_gmv = gmv_weights(Theta_hat)
                weights_dict_by_type['gmv'] = {asset: w_gmv[i] for i, asset in enumerate(current_assets)}
            
            if 'mv' in portfolio_types or 'msr' in portfolio_types:
                # Compute mean returns for MV and MSR
                mu = Y.mean(axis=0)
                
                if 'mv' in portfolio_types:
                    if verbose:
                        print(f"  Computing MV weights (target={mv_target_return})...")
                    w_mv = mv_weights(Theta_hat, mu, target_return=mv_target_return)
                    weights_dict_by_type['mv'] = {asset: w_mv[i] for i, asset in enumerate(current_assets)}
                
                if 'msr' in portfolio_types:
                    if verbose:
                        print(f"  Computing MSR weights...")
                    w_msr = msr_weights(Theta_hat, mu)
                    weights_dict_by_type['msr'] = {asset: w_msr[i] for i, asset in enumerate(current_assets)}
            
        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
                print(f"  Recording zero return for all portfolios")
            
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
            continue

        # Process each portfolio type separately
        for ptype in portfolio_types:
            pdata = portfolio_data[ptype]
            new_weights_dict = weights_dict_by_type[ptype]
            
            # Normalize weights to sum to 1
            weight_sum = sum(new_weights_dict.values())
            if weight_sum > 1e-10:
                new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
            else:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
                continue
            
            # Find common assets
            common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
            
            if len(common_assets) == 0:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
                continue
            
            # Filter and renormalize
            common_weights = {a: new_weights_dict[a] for a in common_assets}
            common_weight_sum = sum(common_weights.values())
            if common_weight_sum > 1e-10:
                common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
            else:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
                continue
            
            # Compute gross return
            gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
            
            # Sanity check
            if np.isnan(gross_return) or np.isinf(gross_return):
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    pdata['prev_weights_dict'], 
                    pdata['prev_oos_returns_dict'], 
                    pdata['prev_gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                pdata['returns'].append(net_return)
                pdata['dates'].append(current_date)
                pdata['weights_list'].append({})
                pdata['turnover_list'].append(turnover)
                pdata['gross_returns'].append(0.0)
                pdata['prev_weights_dict'] = {}
                pdata['prev_oos_returns_dict'] = {}
                pdata['prev_gross_return'] = 0.0
                continue
            
            # Calculate transaction costs
            if len(pdata['prev_weights_dict']) > 0:
                # Adjust previous weights for returns
                adjusted_prev = {}
                for asset, prev_w in pdata['prev_weights_dict'].items():
                    if asset in pdata['prev_oos_returns_dict']:
                        prev_r = pdata['prev_oos_returns_dict'][asset]
                        if abs(1 + pdata['prev_gross_return']) > 1e-6:
                            adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + pdata['prev_gross_return'])
                        else:
                            adjusted_prev[asset] = 0.0
                    else:
                        if abs(1 + pdata['prev_gross_return']) > 1e-6:
                            adjusted_prev[asset] = prev_w / (1 + pdata['prev_gross_return'])
                        else:
                            adjusted_prev[asset] = 0.0
                
                # Calculate turnover
                all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
                turnover = 0.0
                for asset in all_assets:
                    old_w = adjusted_prev.get(asset, 0.0)
                    new_w = common_weights.get(asset, 0.0)
                    turnover += abs(new_w - old_w)
                
                tc = transaction_cost * (1 + gross_return) * turnover
            else:
                # First period
                turnover = sum(abs(w) for w in common_weights.values())
                tc = transaction_cost * (1 + gross_return) * turnover
            
            # Net return
            net_return = gross_return - tc
            
            # Store results
            pdata['returns'].append(net_return)
            pdata['dates'].append(current_date)
            pdata['weights_list'].append(common_weights.copy())
            pdata['turnover_list'].append(turnover)
            pdata['gross_returns'].append(gross_return)
            pdata['prev_weights_dict'] = common_weights.copy()
            pdata['prev_oos_returns_dict'] = {a: oos_returns_dict[a] for a in common_assets}
            pdata['prev_gross_return'] = gross_return
        
        if verbose:
            # Print summary for all portfolios
            for ptype in portfolio_types:
                pdata = portfolio_data[ptype]
                if len(pdata['returns']) > 0:
                    last_return = pdata['returns'][-1]
                    last_gross = pdata['gross_returns'][-1] if len(pdata['gross_returns']) > 0 else 0
                    last_turnover = pdata['turnover_list'][-1] if len(pdata['turnover_list']) > 0 else 0
                    last_tc = last_gross - last_return
                    print(f"  {ptype.upper()}: Gross={last_gross:>8.5f} | TO={last_turnover:>6.4f} | "
                          f"TC={last_tc:>8.6f} | Net={last_return:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results for Each Portfolio ---
    all_results = {}
    
    for ptype in portfolio_types:
        pdata = portfolio_data[ptype]
        
        results_df = pd.DataFrame({
            'date': pdata['dates'],
            'portfolio_return': pdata['returns'],
            'portfolio_gross_return': pdata['gross_returns'],
            'portfolio_weights': pdata['weights_list'],
            'portfolio_turnover': pdata['turnover_list']
        })
        results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
        
        # Compute metrics
        if len(pdata['returns']) > 0:
            mean_return = np.mean(pdata['returns'])
            variance = np.var(pdata['returns'], ddof=1)
            sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
            
            # Annualized metrics (monthly data)
            annual_return = mean_return * 12
            annual_volatility = np.sqrt(variance * 12)
            annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
            
            metrics = {
                'mean_return': mean_return,
                'variance': variance,
                'sharpe_ratio': sharpe_ratio,
                'annual_return': annual_return,
                'annual_volatility': annual_volatility,
                'annual_sharpe_ratio': annual_sharpe,
                'total_return': results_df['cumulative_return'].iloc[-1],
                'avg_turnover': np.mean(pdata['turnover_list']),
                'n_periods': len(pdata['returns']),
                'n_zero_periods': sum(1 for r in pdata['returns'] if r == 0)
            }
        else:
            metrics = {
                'mean_return': 0,
                'variance': 0,
                'sharpe_ratio': 0,
                'annual_return': 0,
                'annual_volatility': 0,
                'annual_sharpe_ratio': 0,
                'total_return': 0,
                'avg_turnover': 0,
                'n_periods': 0,
                'n_zero_periods': 0
            }
        
        all_results[ptype] = (results_df, metrics)
    
    return all_results

In [2]:
# Load your returns data
df = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

In [3]:
# Run backtest with factor-based nodewise regression
all_results = backtest_factor_nodewise_yearly(
    df,
    buys_path_template='novy_marx_buys_{}.csv',
    sells_path_template='novy_marx_sells_{}.csv',
    factors_path='factors_ff_monthly_raw.csv',
    test_start_date='2020-01-31',
    test_end_date='2024-04-30',
    lookback_window=180,
    ic='GIC',  # or 'BIC', 'WIC', 'AIC', 'cv'
    verbose=True
)

# Access individual portfolio results
gmv_results, gmv_metrics = all_results['gmv']
mv_results, mv_metrics = all_results['mv']
msr_results, msr_metrics = all_results['msr']

print("GMV")
print(f"\nSharpe Ratio: {gmv_metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {gmv_metrics['total_return']:.4f}")
print(f"Average Turnover: {gmv_metrics['avg_turnover']:.4f}")

print("MV")
print(f"\nSharpe Ratio: {mv_metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {mv_metrics['total_return']:.4f}")
print(f"Average Turnover: {mv_metrics['avg_turnover']:.4f}")

print("MSR")
print(f"\nSharpe Ratio: {msr_metrics['sharpe_ratio']:.4f}")
print(f"Annualized Sharpe Ratio: {msr_metrics['annual_sharpe_ratio']:.4f}")
print(f"Total Return: {msr_metrics['total_return']:.4f}")
print(f"Average Turnover: {msr_metrics['avg_turnover']:.4f}")

ERROR! Session/line number was not unique in database. History logging moved to new session 709
STARTING FACTOR-BASED BACKTEST: GMV, MV, MSR
Information Criterion: GIC

[1/52] Date: 2020-01-31 | Year: 2020
  Window: 2005-01-31 to 2019-12-31
  Signals: 300 permnos | Assets: 144
  Running Factor-based Nodewise Regression (IC=GIC)...
  Computing GMV weights...
  Computing MV weights (target=0.01)...
  Computing MSR weights...
  GMV: Gross=-0.09267 | TO=2.3059 | TC=0.002092 | Net=-0.09476
  MV: Gross=-0.08845 | TO=2.3357 | TC=0.002129 | Net=-0.09058
  MSR: Gross=-0.06670 | TO=3.5545 | TC=0.003317 | Net=-0.07002

[2/52] Date: 2020-02-29 | Year: 2020
  Window: 2005-02-28 to 2020-01-31
  Signals: 300 permnos | Assets: 144
  Running Factor-based Nodewise Regression (IC=GIC)...
  Computing GMV weights...
  Computing MV weights (target=0.01)...
  Computing MSR weights...
  GMV: Gross=-0.07227 | TO=0.2282 | TC=0.000212 | Net=-0.07249
  MV: Gross=-0.05304 | TO=0.3108 | TC=0.000294 | Net=-0.05333
 

In [4]:
print(f"\n GMV")
print(f"Annualized Sharpe Ratio: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {gmv_metrics['mean_return']*12:.4f}")
print(f"Variance: {gmv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {gmv_metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"Annualized Sharpe Ratio: {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {mv_metrics['mean_return']*12:.4f}")
print(f"Variance: {mv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {mv_metrics['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"Annualized Sharpe Ratio: {msr_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {msr_metrics['mean_return']*12:.4f}")
print(f"Variance: {msr_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {msr_metrics['avg_turnover']:.4f}")


 GMV
Annualized Sharpe Ratio: 0.0250
Mean Return: 0.0036
Variance: 0.0203
Avg Turnover: 0.4142

 MV
Annualized Sharpe Ratio: 0.0416
Mean Return: 0.0058
Variance: 0.0192
Avg Turnover: 0.4424

 MSR
Annualized Sharpe Ratio: 0.0833
Mean Return: 0.0176
Variance: 0.0447
Avg Turnover: 1.1522
